## import pre-process categoried data and delete function words 

In [177]:
import numpy as np
import pandas as pd
import spacy
import re  ## regexp
import json

In [178]:
import matplotlib.pyplot as plt
import seaborn as sns

In [180]:
import os
dir = os.path.join(os.getcwd())
bus = open(os.path.join(dir,'Data','business_city.json'),encoding="utf8")
bus = pd.DataFrame([json.loads(x) for x in bus.readlines()])

In [3]:
merge = pd.read_csv('data/reviews.csv')
merge.shape

(253757, 4)

In [4]:
merge['text'] = merge.apply(lambda x: x['text'].lower(), axis=1)

In [6]:
merge.head(2)

,review_id,business_id,stars,text
0,LG2ZaYiOgpr2DK_90pYjNw,HQl28KMwrEKHqhFrrDqVNQ,5,i love deagan's. i do. i really do. the atmosp...
1,2mLlQ9n5Ln_fykOuX_b0-A,wSAGILK2CFbq5BVZ5D67-A,3,the service is attentive. the decor attractive...


In [5]:
merge['text'][0]

"i love deagan's. i do. i really do. the atmosphere is cozy and festive. the shrimp tacos and house fries are my standbys. the fries are sometimes good and sometimes great, and the spicy dipping sauce they come with is to die for. the beer list is amazing and the cocktails are great. the prices are mid-level, so it's not a cheap dive you can go to every week, but rather a treat when you do. try it out. you won't be disappointed!"

In [7]:
#print(set(stopwords.words('english')))
nlp = spacy.load("en")
nlp.Defaults.stop_words |= {".","'","[","]","(",")"}

In [8]:
print(nlp.Defaults.stop_words)

{'elsewhere', 'somehow', 'each', 'meanwhile', 'call', 'of', 'less', 'we', '‘m', 'i', 'another', "'d", 'go', 'someone', 'an', 'too', 'everything', 'latter', '’re', 'almost', 'anywhere', 'anything', 'mine', 'whereupon', 'being', 'must', 'five', 'formerly', 'still', 'my', 'done', 'say', 'hereupon', 'neither', ']', 'serious', 'for', 'yours', 'everyone', 'third', "'re", 'own', 'next', 'without', 'no', "n't", 'front', 'nor', 'may', 'through', 'thru', 'as', 'first', 'former', 'take', 'fifty', 'forty', 'although', 'either', 'anyway', 'besides', 'had', '.', 'due', 'across', 'about', 'often', 'whence', 'last', 'around', 'from', 'nowhere', 'else', 'fifteen', 'ten', 'nine', 'towards', 'our', 'alone', 'give', 'nevertheless', 'into', 'otherwise', 'and', 'against', 'by', 'thereupon', 'nothing', 'seeming', 'show', 'ca', 'very', 'one', 'have', 'bottom', 'here', 'up', 'afterwards', 'few', 'were', 'full', 'wherever', '‘s', 'seemed', 'ours', 'also', 'you', 'hers', 'only', 'myself', 'quite', 'unless', 'bet

In [9]:
def stop(sentence):
    res = list(filter(lambda x: False if x.is_stop else True, nlp(sentence)))
    return res

In [10]:
merge['text'] = merge['text'].apply(stop)

In [11]:
merge.head(2)

,review_id,business_id,stars,text
0,LG2ZaYiOgpr2DK_90pYjNw,HQl28KMwrEKHqhFrrDqVNQ,5,"[love, deagan, atmosphere, cozy, festive, shri..."
1,2mLlQ9n5Ln_fykOuX_b0-A,wSAGILK2CFbq5BVZ5D67-A,3,"[service, attentive, decor, attractive, menu, ..."


In [12]:
import nltk
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [13]:
def retrive_text(sentence):
    sentence = list(map(lambda x: x.text,sentence))
    return sentence
def form_word(sentence):
    sentence = list(map(lambda x: wnl.lemmatize(x,'v'),sentence))
    sentence = list(map(lambda x: wnl.lemmatize(x,'n'),sentence))
    sentence = list(map(lambda x: wnl.lemmatize(x,'a'),sentence))
    return sentence

In [17]:
merge['text'][0]

['love',
 'deagan',
 'atmosphere',
 'cozy',
 'festive',
 'shrimp',
 'tacos',
 'house',
 'fries',
 'standbys',
 'fries',
 'good',
 'great',
 ',',
 'spicy',
 'dipping',
 'sauce',
 'come',
 'die',
 'beer',
 'list',
 'amazing',
 'cocktails',
 'great',
 'prices',
 'mid',
 '-',
 'level',
 ',',
 'cheap',
 'dive',
 'week',
 ',',
 'treat',
 'try',
 'wo',
 'disappointed',
 '!']

## merge words into one sentence

In [18]:
import torch
from torchtext import data
from torchtext.data import TabularDataset
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',lower=True)
LABEL = data.LabelField(dtype = torch.float)

C:\Users\27477\anaconda3\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
C:\Users\27477\anaconda3\lib\site-packages\torchtext\data\field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [19]:
inte = merge[['text','stars']]
inte

,text,stars
0,"[love, deagan, atmosphere, cozy, festive, shri...",5
1,"[service, attentive, decor, attractive, menu, ...",3
2,"[furry, watching, 2011, concluded, tonic, .......",3
3,"[food, use, fuckn, good, \n, .., suckx, food, ...",3
4,"[great, place, !, place, sushi, cle, tuna, app...",4
...,...,...
253752,"[club, awesome, drink, deals, games, instance​...",4
253753,"[pretty, standard, weekend, brunch, complain, ...",4
253754,"[owner, business, reading, ,, seriously, quest...",1
253755,"[let, beat, bush, ......, um, ..., pun, intend...",4


In [20]:
def list_to_string(sentence):
    sentence = " ".join(sentence)
    return sentence

In [21]:
list(map(list_to_string,inte['text']))

['love deagan atmosphere cozy festive shrimp tacos house fries standbys fries good great , spicy dipping sauce come die beer list amazing cocktails great prices mid - level , cheap dive week , treat try wo disappointed !',
 'service attentive decor attractive menu far doused polenta paprika fingerling potatoes ? drenched paprika aioli?mixed paprika appetizer ? fried polenta fish ? polenta crusted worth try forewarned chef shop costco',
 'furry watching 2011 concluded tonic .... fun time .... furries humans alike \n\n mel u.   past place couple times thought   think " outskirts " town section  \n\n love outdoor seating area !   row seat furry watching action going outside westin convention center   plus let face , great outdoor seating !  \n\n known martini indulge !   ordered cosmo - tonic reason cosmo lately   tini , right mixture   hate cranberry vodka gasp , know !     perfect mix glad report !   nice job shake masters !   tried friends lemon drop martini , holy bejebus , way sweet 

In [22]:
inte['text'] = list(map(list_to_string,inte['text']))

<ipython-input-22-598dd1e83fba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inte['text'] = list(map(list_to_string,inte['text']))


In [23]:
inte.head()

,text,stars
0,love deagan atmosphere cozy festive shrimp tac...,5
1,service attentive decor attractive menu far do...,3
2,furry watching 2011 concluded tonic .... fun t...,3
3,food use fuckn good \n .. suckx food picked ee...,3
4,great place ! place sushi cle tuna appetizer s...,4


In [24]:
inte.to_csv('inte.csv',index=False)

## choose the most frequency words

### besides, it is also needed to delete some function words by hand

In [67]:
txt_datafields = [("text", TEXT), # we won't be needing the id, so we pass in None as the field
                  ("stars", LABEL)]
txt = TabularDataset(
           path="Data/inte.csv", # the file path
           format='csv',
           skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
           fields=txt_datafields)

C:\Users\27477\anaconda3\lib\site-packages\torchtext\data\example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
C:\Users\27477\anaconda3\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [68]:
print(f'Number of testing examples: {len(txt)}')

Number of testing examples: 253757


In [69]:
TEXT.build_vocab(txt)
LABEL.build_vocab(txt)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 111638
Unique tokens in LABEL vocabulary: 5


In [70]:
common = TEXT.vocab.freqs.most_common(514)
total_count = pd.DataFrame(common,columns=['word','count'])

In [71]:
total_count.head(120)

,word,count
0,",",1035177
1,,405842
2,!,298893
3,\n\n,207264
4,food,185847
...,...,...
115,taste,17138
116,fun,17006
117,quality,16958
118,friend,16929


In [72]:
total_count.drop(index = [0,1,2,3,6,10,14,17,22,24,48,62,81,87,94,96,107],inplace = True)

In [73]:
total_count.head()

,word,count
4,food,185847
5,good,165284
7,place,142852
8,great,138515
9,service,95733


In [74]:
inte.shape

(253757, 2)

In [75]:
TEXT.build_vocab(txt)
LABEL.build_vocab(txt)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 111638
Unique tokens in LABEL vocabulary: 5


In [76]:
total_count.shape

(497, 2)

## extra cleaning

In [80]:
import json
import string

In [89]:
text = merge[['text','stars','business_id']]
review_text = text.loc[:,'text'].tolist()

In [90]:
text.head()

,text,stars,business_id
0,"[love, deagan, atmosphere, cozy, festive, shri...",5,HQl28KMwrEKHqhFrrDqVNQ
1,"[service, attentive, decor, attractive, menu, ...",3,wSAGILK2CFbq5BVZ5D67-A
2,"[furry, watching, 2011, concluded, tonic, .......",3,nkDSE-yhvLX4ij5fSzvb5Q
3,"[food, use, fuckn, good, \n, .., suckx, food, ...",3,X2f2s37_cFKo4xpzGFqk1w
4,"[great, place, !, place, sushi, cle, tuna, app...",4,APPZiUz_CEvR_--qY-tbig


In [96]:
merge['text']

0         [love, deagan, atmosphere, cozy, festive, shri...
1         [service, attentive, decor, attractive, menu, ...
2         [furry, watching, 2011, concluded, tonic, .......
3         [food, use, fuckn, good, \n, .., suckx, food, ...
4         [great, place, !, place, sushi, cle, tuna, app...
                                ...                        
253752    [club, awesome, drink, deals, games, instance​...
253753    [pretty, standard, weekend, brunch, complain, ...
253754    [owner, business, reading, ,, seriously, quest...
253755    [let, beat, bush, ......, um, ..., pun, intend...
253756    [staff, friendly, helpful, food, says, burgers...
Name: text, Length: 253757, dtype: object

In [108]:
review_text = text.loc[:,'text'].tolist()

In [129]:
remove = string.punctuation
remove = remove.replace(',','')
def convert_data(data):
  data = re.sub("[{}]+".format(remove), '',str(data))
  data = re.sub(' +',",",str(data))
  data = data.split(',')
  return data

In [130]:
review.append(convert_data(review_text[i]))

In [131]:
review = []
for i in range(len(text)):
  review.append(convert_data(review_text[i]))

In [134]:
text['text'] = review

<ipython-input-134-df7aef1b0784>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['text'] = review


In [135]:
text['text'][0]

['love',
 'deagan',
 'atmosphere',
 'cozy',
 'festive',
 'shrimp',
 'tacos',
 'house',
 'fries',
 'standbys',
 'fries',
 'good',
 'great',
 'spicy',
 'dipping',
 'sauce',
 'come',
 'die',
 'beer',
 'list',
 'amazing',
 'cocktails',
 'great',
 'prices',
 'mid',
 'level',
 'cheap',
 'dive',
 'week',
 'treat',
 'try',
 'wo',
 'disappointed',
 '']

In [137]:
text.head()

,text,stars,business_id
0,"[love, deagan, atmosphere, cozy, festive, shri...",5,HQl28KMwrEKHqhFrrDqVNQ
1,"[service, attentive, decor, attractive, menu, ...",3,wSAGILK2CFbq5BVZ5D67-A
2,"[furry, watching, 2011, concluded, tonic, fun,...",3,nkDSE-yhvLX4ij5fSzvb5Q
3,"[food, use, fuckn, good, \n, suckx, food, pick...",3,X2f2s37_cFKo4xpzGFqk1w
4,"[great, place, place, sushi, cle, tuna, appeti...",4,APPZiUz_CEvR_--qY-tbig


### negative word import

In [98]:
import os
os.getcwd()

'C:\\Users\\27477\\Desktop\\628HW3'

In [104]:
neg_word = pd.read_table('./negative_words.txt',sep='\t',header=None,names=['word'])
neg_word = neg_word.loc[:,'word'].values.tolist()
print(neg_word)

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted', 'aborts', 'abrade', 'abrasive', 'abrupt', 'abruptly', 'abscond', 'absence', 'absent-minded', 'absentee', 'absurd', 'absurdity', 'absurdly', 'absurdness', 'abuse', 'abused', 'abuses', 'abusive', 'abysmal', 'abysmally', 'abyss', 'accidental', 'accost', 'accursed', 'accusation', 'accusations', 'accuse', 'accuses', 'accusing', 'accusingly', 'acerbate', 'acerbic', 'acerbically', 'ache', 'ached', 'aches', 'achey', 'aching', 'acrid', 'acridly', 'acridness', 'acrimonious', 'acrimoniously', 'acrimony', 'adamant', 'adamantly', 'addict', 'addicted', 'addicting', 'addicts', 'admonish', 'admonisher', 'admonishingly', 'admonishment', 'admonition', 'adulterate', 'adulterated', 'adulteration', 'adulterier', 'adversarial', 'adversary', 'adverse', 'adversity', 'afflict', 'affliction', 'afflictive', 'affront', 'afraid', 'aggravate', 'aggravating', 'aggravation', 'aggression', 'aggress

### possitive word import

In [105]:
pos_word = pd.read_csv('./positive_words.txt',sep='\t',header=None,names=['word'])
pos_word = pos_word.loc[:,'word'].values.tolist()
print(pos_word)

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'ag

In [136]:
text['text'][0]

['love',
 'deagan',
 'atmosphere',
 'cozy',
 'festive',
 'shrimp',
 'tacos',
 'house',
 'fries',
 'standbys',
 'fries',
 'good',
 'great',
 'spicy',
 'dipping',
 'sauce',
 'come',
 'die',
 'beer',
 'list',
 'amazing',
 'cocktails',
 'great',
 'prices',
 'mid',
 'level',
 'cheap',
 'dive',
 'week',
 'treat',
 'try',
 'wo',
 'disappointed',
 '']

## count for positive and negative words

In [138]:
business_id = list(text.business_id.unique())

In [139]:
def find_adjacent(noun, review):
  pos_label = [99999]
  neg_label = [99999]
  noun_label = []
  count = 0
  for index, word in enumerate(review):
    if word in pos_word:
      pos_label.append(index)
    if word in neg_word:
      neg_label.append(index)
    if noun == word:
      noun_label.append(index)
  for index in noun_label:
    if min(abs(np.array(pos_label)-index)) < min(abs(np.array(neg_label)-index)):
      count += 1
  return np.array((count,len(noun_label)-count))

In [140]:
def pos_neg_count(noun, review):
  count_result = []
  for word in noun:
    num = np.array((0,0))
    for i in range(len(review)):
      num += find_adjacent(word,review[i])
    count_result.append([word, num[0], num[1]])
  return pd.DataFrame(count_result,columns=['word','pos_num','neg_num'])

### selected 'part' words by hand (in order to count sentiment words by three part, price, service and flavour)

In [149]:
total_count.head(80)

,word,count
4,food,185847
5,good,165284
7,place,142852
8,great,138515
9,service,95733
...,...,...
89,asked,21183
90,friends,21115
91,awesome,21084
92,tried,21050


In [150]:
specificfood=['cheese','chicken','burger','sushi','fries','sauce','salad','sandwich','pizza']
flavour = ['food','bar', 'menu','beer','restaurant','dinner','drink','meal']
price = ['ordered','order','table']
service = ['place','service','time','night','staff','people','experience','atmosphere','wait','server','area','minutes','lunch']

In [154]:
def business_pos_neg_count(specificfood,flavour, price, service, business_id):
  result = pd.DataFrame(columns = ['word','pos_num','neg_num','subject','business_id','review_num'])
  for id in business_id:
    temp = text.loc[text['business_id'] == id]
    specificfood_result = pos_neg_count(specificfood,temp['text'].tolist())
    specificfood_result ['subject'] = 'specificfood'
    flavour_result = pos_neg_count(flavour,temp['text'].tolist())
    flavour_result['subject'] = 'flavour'
    price_result = pos_neg_count(price,temp['text'].tolist())
    price_result['subject'] = 'price'
    service_result = pos_neg_count(service,temp['text'].tolist())
    service_result['subject'] = 'service'
    temp_result = pd.concat([specificfood_result,flavour_result,price_result,service_result])
    temp_result['business_id'] = id
    temp_result['review_num'] = len(temp)
    result = pd.concat([result,temp_result])
  return result

In [155]:
result = business_pos_neg_count(specificfood,flavour, price, service, business_id) ## cost a definitely long time

In [156]:
result.shape

(119790, 6)

In [190]:
state = bus[['business_id','state']]
state.head()

,business_id,state
0,pQeaRpvuhoEqudo3uymHIQ,IL
1,7Dv4_HAxsxvadEsT5fxQBg,PA
2,M_guz7Dj7hX0evS672wIwA,WI
3,03x6ZlJ7s39DHqfTJU1Ecg,OH
4,_4Oe9V-qTpU5iemM9bphlA,PA


In [188]:
result.head()

,word,pos_num,neg_num,subject,business_id,review_num
0,cheese,138,50,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453
1,chicken,154,51,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453
2,burger,65,29,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453
3,sushi,1,0,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453
4,fries,93,25,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453


In [189]:
sub1 = pd.DataFrame(result, columns = ['word','pos_num','neg_num','subject','business_id','review_num']) 

In [191]:
sub1 = sub1.join(state.set_index('business_id'), on='business_id')
sub1.head()

,word,pos_num,neg_num,subject,business_id,review_num,state
0,cheese,138,50,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453,OH
1,chicken,154,51,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453,OH
2,burger,65,29,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453,OH
3,sushi,1,0,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453,OH
4,fries,93,25,specificfood,HQl28KMwrEKHqhFrrDqVNQ,453,OH


In [192]:
sub1.to_csv('C:\\Users\\27477\\Desktop\\628HW3\\business_pos_neg.csv',index=False,header=True)